In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-16 02:23:14--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-08-16 02:23:14 (6.56 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...| 2015-08-31|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...|           Shoes|          5|    

Step 1


In [6]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes 
# count is equal to or greater than 20 
votes_df = df.filter("total_votes>=20")
votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   52181093|R37F42INKX7L9K|B00TQ6L9TK|     316459194|Travelon Converti...|           Shoes|          5|           45|         49|   N|                Y|Hermione Granger'...|I've been trying ...| 2015-08-31|
|         US|   25865070|R2EHKYNEP8WVSR|B00UJ6B33I|     962947795|Hoxis Tassel Faux...|           Shoes|          5|    

Step 2

In [9]:
# retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater 
# than 50% 
helpful_divide_total_df= votes_df.filter("(helpful_votes/total_votes)>=0.5")
helpful_divide_total_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   52181093|R37F42INKX7L9K|B00TQ6L9TK|     316459194|Travelon Converti...|           Shoes|          5|           45|         49|   N|                Y|Hermione Granger'...|I've been trying ...| 2015-08-31|
|         US|   25865070|R2EHKYNEP8WVSR|B00UJ6B33I|     962947795|Hoxis Tassel Faux...|           Shoes|          5|    

Step 3

In [10]:
# Retrieves all the rows where a review was written as part of the Vine program (paid)
Review_was_written_df = helpful_divide_total_df.filter("vine == 'Y'")
Review_was_written_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   17481726|R2N45ZKRRZS856|B00R9LA030|     587914328|New Balance Women...|           Shoes|          5|           21|         22|   Y|                N|Lightweight, Plen...|* While I've got ...| 2015-07-19|
|         US|   34395794| R5OMLMK13A8NS|B00R9LA030|     587914328|New Balance Women...|           Shoes|          5|    

Step 4

In [11]:
# Review was not written 
Review_wasnt_written_df = helpful_divide_total_df.filter("vine == 'N'")
Review_wasnt_written_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   52181093|R37F42INKX7L9K|B00TQ6L9TK|     316459194|Travelon Converti...|           Shoes|          5|           45|         49|   N|                Y|Hermione Granger'...|I've been trying ...| 2015-08-31|
|         US|   25865070|R2EHKYNEP8WVSR|B00UJ6B33I|     962947795|Hoxis Tassel Faux...|           Shoes|          5|    

Step 5

In [ ]:
# Determine the total number of reviews, the number of 5-star reviews, and the precentage of 5-star 
# reviews for the two types of review 


In [14]:
# Total number of reviews
total_number_reviews = helpful_divide_total_df.count()
total_number_reviews

27009

In [15]:
# Total paid reviews
total_paid_reviews = Review_was_written_df.count()
total_paid_reviews

22

In [16]:
# Total unpaid reviews
total_unpaid_reviews = Review_wasnt_written_df.count()
total_unpaid_reviews

26987

In [17]:
# Total 5 stars reviews
total_five_star_reviews = helpful_divide_total_df.filter("star_rating == 5").count()
total_five_star_reviews

14488

In [19]:
# Total 5 stars paid reviews
total_five_star_paid_reviews = Review_was_written_df.filter("star_rating == 5").count()
total_five_star_paid_reviews

13

In [20]:
# Total 5 stars unpaid reviews
total_five_star_unpaid_reviews = Review_wasnt_written_df.filter("star_rating == 5").count()
total_five_star_unpaid_reviews

14475

In [23]:
# percentage of 5 stars in paid reviews 
100*total_five_star_paid_reviews/total_paid_reviews


59.09090909090909

In [24]:
# percentage of 5 stars in unpaid reviews 
100*total_five_star_unpaid_reviews/total_unpaid_reviews

53.636936302664246